<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/EMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# EMNIST

EMNIST is an extention of MNIST dataset. It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

In [0]:
from torchvision import datasets, transforms
import torch

train_dataset = datasets.EMNIST('/data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

test_dataset = datasets.EMNIST('../data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

#1

Find correspondence between classess and letters/digits

In [0]:
letters = train_dataset.classes_split_dict['letters']
digits  = train_dataset.classes_split_dict['digits']
byclass = train_dataset.classes_split_dict['byclass']  # if not why not
balanced = train_dataset.classes_split_dict['balanced']

In [3]:
len(balanced)

47

#2

Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can.\
Explain, what experiments have you perform.\
What optimizers have you tested?\
How network/optimizer parameters were chosen.

In [0]:
import torch.nn as nn
from torch.nn.modules import loss

class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 64)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [0]:
learning_rate = 0.001 # Small enough
num_epochs = 25 #If not why not
num_classes = len(balanced)
batch_size = 100

In [0]:
model = Net()

device    = torch.device("cuda")
# device    = torch.device("cpu")

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

optimizer_1 = torch.optim.Adam(
                    model.parameters(), 
                    lr=learning_rate)  

optimizer = torch.optim.Adagrad(
                    model.parameters(),
                    lr = learning_rate, 
                    lr_decay = 0.01)
scheduler = torch.optim.lr_scheduler.StepLR(
                                            optimizer,
                                            0.5)

In [0]:
def test_ass(model, test_loader, loss_function, device = device):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
    
            output = model(data)
    
            test_loss += loss_function(output, target).sum().item()
    
            pred = output.argmax(dim=1, keepdim=True)
    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_ass(model, train_loader, optimizer, loss_function, epoch , device = device):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = loss_function(output, target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
for epoch in range(20):
    train_ass(model, train_loader, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass(model, test_loader, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/112800 (0%)]	Loss: 4.155716
Train Epoch: 0 [32000/112800 (28%)]	Loss: 1.055035
Train Epoch: 0 [64000/112800 (57%)]	Loss: 0.890459
Train Epoch: 0 [96000/112800 (85%)]	Loss: 0.776751

Test set: Average loss: 0.0155, Accuracy: 13457/18800 (72%)

Train Epoch: 1 [0/112800 (0%)]	Loss: 1.485131
Train Epoch: 1 [32000/112800 (28%)]	Loss: 1.204287
Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.910169
Train Epoch: 1 [96000/112800 (85%)]	Loss: 0.917098

Test set: Average loss: 0.0151, Accuracy: 13561/18800 (72%)

Train Epoch: 2 [0/112800 (0%)]	Loss: 1.197139
Train Epoch: 2 [32000/112800 (28%)]	Loss: 1.153221
Train Epoch: 2 [64000/112800 (57%)]	Loss: 1.021487
Train Epoch: 2 [96000/112800 (85%)]	Loss: 0.844010

Test set: Average loss: 0.0150, Accuracy: 13601/18800 (72%)

Train Epoch: 3 [0/112800 (0%)]	Loss: 1.303065
Train Epoch: 3 [32000/112800 (28%)]	Loss: 0.996297
Train Epoch: 3 [64000/112800 (57%)]	Loss: 1.142824
Train Epoch: 3 [96000/112800 (85%)]	Loss: 1.071908

Test set: Average

Adam, lr (0.01) >> Accuracy of 88%\
Adagrad, lr (0.01) >> Accuracy of 73%\
 .

#3

Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model)

*MNIST*

In [0]:
from torchvision import datasets, transforms
import torch

train_dataset_mnist = datasets.MNIST(
                                '/data', 
                                train = True, 
                                download = True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
test_dataset_mnist = datasets.MNIST(
                                '../data', 
                                train = False, 
                                download = True,
                                transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
train_loader_mnist = torch.utils.data.DataLoader(
                                        train_dataset_mnist, 
                                        batch_size=64, 
                                        shuffle=True)

test_loader_mnist = torch.utils.data.DataLoader(
                                        test_dataset_mnist, 
                                        batch_size=64, 
                                        shuffle=True)

In [0]:
class MNISTNet(nn.Module):
    
    def __init__(self):
        
        super(MNISTNet, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10) # changed from self.fc2 = nn.Linear(1000, 64)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model_mnist = MNISTNet()

In [12]:
model_mnist = model.to(device)

optimizer_mnist_1 = torch.optim.Adam(
                    model_mnist.parameters(), 
                    lr=learning_rate)  

optimizer_mnist = torch.optim.Adagrad(
                    model_mnist.parameters(),
                    lr = learning_rate, 
                    lr_decay = 0.01)

scheduler = torch.optim.lr_scheduler.StepLR(
                                            optimizer_mnist,
                                            0.5)
for epoch in range(20):
    train_ass(model_mnist, train_loader_mnist, optimizer_mnist, loss.CrossEntropyLoss(), epoch)    
    test_ass(model_mnist, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 7.591644
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.249015

Test set: Average loss: 0.0024, Accuracy: 9581/10000 (96%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.081704
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.175206

Test set: Average loss: 0.0022, Accuracy: 9603/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.303017
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.325987

Test set: Average loss: 0.0021, Accuracy: 9612/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.289603
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.102545

Test set: Average loss: 0.0021, Accuracy: 9618/10000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.309292
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.269517

Test set: Average loss: 0.0021, Accuracy: 9622/10000 (96%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.149467
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.112939

Test set: Average loss: 0.0021, Accuracy: 9624/10000 (96%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.195254
Train Epoch: 6 [320

In [14]:
for epoch in range(20):
    train_ass(model_mnist, train_loader_mnist, optimizer_mnist_1, loss.CrossEntropyLoss(), epoch)    
    test_ass(model_mnist, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 12.542337
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.042926

Test set: Average loss: 0.0006, Accuracy: 9866/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.035255
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.082642

Test set: Average loss: 0.0005, Accuracy: 9892/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.014404
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.022710

Test set: Average loss: 0.0006, Accuracy: 9881/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.007643
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.017210

Test set: Average loss: 0.0005, Accuracy: 9902/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.067244
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.184245

Test set: Average loss: 0.0005, Accuracy: 9899/10000 (99%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.020625
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.070794

Test set: Average loss: 0.0005, Accuracy: 9911/10000 (99%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.082387
Train Epoch: 6 [32

Adagrad(96%)\
Adam(99%)


#4

Use EMNIST classifier to classify MNIST test dataset.\

* To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value 

(\
    torch tensors can be indexed with lists, like this:\
    --> torch.tensor(
        [
            [1,2],
            [3,4], 
            [5, 6]
        ])[[0, 1]]\
).\
\
Which of the models performs better?

In [0]:
def test_ass_2(model, test_loader, loss_function, device = device):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
    
            output = model(data)
    
            test_loss += loss_function(output[:,0:10], target).sum().item()
    
            pred = output[:,0:10].argmax(dim=1, keepdim=True)
    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_ass_2(model, train_loader, optimizer, loss_function, epoch , device = device):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = loss_function(output[:,0:10], target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

for epoch in range(20):
    train_ass_2(model, train_loader_mnist, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass_2(model, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

In [20]:
for epoch in range(20):
    train_ass_2(model, train_loader_mnist, optimizer_1, loss.CrossEntropyLoss(), epoch)    
    test_ass_2(model, test_loader_mnist, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.008963
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.013973

Test set: Average loss: 0.0008, Accuracy: 9909/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.127903
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.000499

Test set: Average loss: 0.0008, Accuracy: 9909/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.032082
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.002516


KeyboardInterrupt: ignored

Adam(99%)\
Adagrad(99%)

EMNIST is an extention of MNIST dataset.
It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

1. Find correspondence between classess and letters/digits.
2. Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can. Explain, what experiments have you perform. What optimizers have you tested? How network/optimizer parameters were chosen.
3. Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model).
4. Use EMNIST classifier to classify MNIST test dataset. To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value (torch tensors can be indexed with lists, like this: `torch.tensor([[1,2],[3,4], [5, 6]])[[0, 1]]`). Which of the models performs better?
5. Present a short (2-3 pages long) report on this experiment. What approaches have you tried? How can you explain the results you got? Include tensorboard plots in the experiment report.
